In [6]:
import tweepy
import pandas as pd
import time

# Authentication with Twitter API V2
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAADcsxQEAAAAA1FJXhrBL%2FXo3Nlw78YC5TVO4jns%3D30eM1mRWzb9erLFlvSrqupvdS9pByR6xE530Vn6gJmK26ychGb"

# Set up Tweepy client for API V2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Define split search queries
query_parts = [
    '"Kosovo" OR "Kosova" OR "Serbia" OR "Serb" OR "Albanians" OR "Serbs"',
    '"#Kosovo" OR "#Serbia" OR "#KosovoSerbia" OR "#KosovoConflict" OR "#KosovoIndependence"',
    '"terrorist" OR "occupier" OR "enemy" OR "traitor" OR "backward" OR "barbaric"',
    '"genocide" OR "ethnic" OR "attack" OR "war crimes"',
    '"Kosova" OR "shqiptarë" OR "serbë" OR "pavarësi" OR "luftë"',
    '"Independence" OR "Albanian" OR "KosovoAlbanians" OR "Srbi" OR "Albanci"',
    '"nezavisnost" OR "rat" OR "SerbiaKosovo" OR "SerbNationalism" OR "SerbianPride"',
    '"KosovoSerbs" OR "NATO" OR "United Nations" OR "peace agreement"',
    '"territory" OR "self-determination" OR "sovereignty" OR "border dispute"',
    '"international law" OR "KosovoLiberationArmy" OR "Yugoslavia" OR "Kosovo War"',
    '"1999" OR "SerbiaKosovoConflict" OR "military" OR "peace talks" OR "UN Resolution" OR "ethnic cleansing"'
]

# Initialize variables
tweets_data = []
max_tweets = 100  # Set the total number of tweets you want to retrieve
tweet_count = 0
next_token = None

# Loop through each query part
for query in query_parts:
    while tweet_count < max_tweets:
        # Make the request to search recent tweets
        try:
            if next_token:
                tweets = client.search_recent_tweets(query=query, max_results=100, next_token=next_token)
            else:
                tweets = client.search_recent_tweets(query=query, max_results=100)
            
            # Extract tweet details and store in list
            for tweet in tweets.data:
                tweets_data.append([tweet.created_at, tweet.author_id, tweet.text])
                tweet_count += 1
            
            # Check if there's a next_token to continue pagination
            next_token = tweets.meta.get('next_token') if 'next_token' in tweets.meta else None
            
            # Break the loop if there are no more tweets
            if not next_token:
                break
        
        except tweepy.TooManyRequests as e:
            # Wait for the rate limit window to reset
            print("Rate limit exceeded, waiting for 15 minutes...")
            time.sleep(15 * 60)  # Wait for 15 minutes

# Create DataFrame
df = pd.DataFrame(tweets_data, columns=["Date", "User", "Tweet"])

# Save DataFrame to CSV
df.to_csv("kosovo_serbia_tweets.csv", index=False)

print(f"Total tweets collected: {tweet_count}")
print(df.head())


Rate limit exceeded, waiting for 15 minutes...


KeyboardInterrupt: 